In [68]:

import pandas as pd
import numpy as np


#!pip install geopandas
#!pip install geopy
#!pip install folium

import folium
import geopandas
import geopy
from folium.plugins.marker_cluster import MarkerCluster
import folium.plugins as plugins
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [51]:
# Test
locator = Nominatim(user_agent='myGeocoder')
location = locator.geocode('Pladehals Allé, Copenhagen, Denmark')

#print('Latitude = {}, Longitude = {}'.format(location.latitude, location.longitude))

df = pd.read_csv('addresses.csv')
df.head()

,Unnamed: 0,Typ,Nr,Namn,Address1,Address3,Address4,Address5,Telefon
0,0,Butik,102,Fältöversten,Karlaplan 13,115 20,STOCKHOLM,Stockholms län,08/662 22 89
1,1,Butik,104,NaN,Nybrogatan 47,114 39,STOCKHOLM,Stockholms län,08/662 50 16
2,2,Butik,106,Garnisonen,Karlavägen 100 A,115 26,STOCKHOLM,Stockholms län,08/662 64 85
3,3,Butik,110,NaN,Hötorgshallen,111 57,STOCKHOLM,Stockholms län,08/56849241
4,4,Butik,113,Sergel,Drottninggatan 45,111 21,STOCKHOLM,Stockholms län,08/21 47 44


In [52]:
df1 =df

df1["ADDRESS"] = df1['Address1'].astype(str) + ", " + df1['Address3'].astype(str) + "," + df1['Address4'].astype(str) + "," + df1['Address5'].astype(str) + ", " + "Sweden"
df1.head()

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df1['location'] = df1['ADDRESS'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df1['point'] = df1['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df1[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df1['point'].tolist(), index=df.index)

In [58]:
#df1 = df1.drop(['ADDRESS', 'Address3', 'Address4', 'Address5','Telefon', 'ADDRESS', 'location', 'point','Namn'], axis=1)
df1 = df1.dropna()
df1.head(30)

,Unnamed: 0,Typ,Nr,Address1,latitude,longitude,altitude
0,0,Butik,102,Karlaplan 13,59.338877,18.090865,0.0
1,1,Butik,104,Nybrogatan 47,59.337207,18.079098,0.0
2,2,Butik,106,Karlavägen 100 A,59.335673,18.099179,0.0
3,3,Butik,110,Hötorgshallen,59.334327,18.062604,0.0
4,4,Butik,113,Drottninggatan 45,59.333613,18.061674,0.0
5,5,Butik,114,Norrlandsgatan 3,59.333177,18.071281,0.0
6,6,Butik,116,Drottninggatan 22,59.330467,18.064907,0.0
7,7,Butik,132,Rålambsvägen 7-9,59.327990,18.017411,0.0
8,8,Butik,133,Kungsholmstorg 11 A,59.328761,18.043296,0.0
9,9,Butik,134,Lindhagensgatan 118,59.337281,18.009324,0.0


In [67]:
map1 = folium.Map(
    location=[59.338315,18.089960],
    tiles='CartoDB dark_matter',
    zoom_start=12,
)
df1.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1